## Lab 3: Part 2
### Comparing Methods for Interpolation of Temperature Data

This lab aims to produce interpolated temperature data from NDAWN station locations using three different methods for calucating interpolated values. The current temperature data is requested from the NDAWN API, producing an on-the-fly (realtime) temperature interpolation map.

### 0. Import Libraries and Packages

In [1]:
# Import Libraries
import io
import os
import sys
import arcpy
from zipfile import ZipFile
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureCollection, FeatureSet
from arcgis.geometry import SpatialReference

### 1. Extract the Weather Data and Transform the Dataframe

In [32]:
# Request: NDAWN Data
ndawn_link = r"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=229&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=228&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d&begin_date=2024-11-11&end_date=2024-11-11"
ndawn_request = requests.get(ndawn_link)
ndawn_response = ndawn_request.content

#print(ndawn_response)
df = pd.read_csv(io.StringIO(ndawn_response.decode('utf-8')), skiprows=[0,1,2,4], index_col=False)
df.drop(df.columns[[8,10,12]], axis=1, inplace=True)
df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Min Temp,Avg Temp
0,Ada,47.32119,-96.51406,910,2024,10,13,52.466,32.067,42.267
1,Ada,47.32119,-96.51406,910,2024,10,14,48.560,26.742,37.651
2,Ada,47.32119,-96.51406,910,2024,10,15,54.302,21.000,37.651
3,Ada,47.32119,-96.51406,910,2024,10,16,63.572,38.008,50.790
4,Ada,47.32119,-96.51406,910,2024,10,17,70.232,51.008,60.620
...,...,...,...,...,...,...,...,...,...,...
6501,Zeeland,46.01351,-99.68768,2070,2024,11,7,50.000,21.344,35.672
6502,Zeeland,46.01351,-99.68768,2070,2024,11,8,56.228,23.007,39.618
6503,Zeeland,46.01351,-99.68768,2070,2024,11,9,46.994,31.284,39.139
6504,Zeeland,46.01351,-99.68768,2070,2024,11,10,53.816,32.974,43.395


In [36]:
# Group by 'Station Name', 'Latitude', 'Longitude', 'Elevation' and compute the mean of Max Temp, Min Temp, and Avg Temp
mean_df = (
    df.groupby(['Station Name', 'Latitude', 'Longitude', 'Elevation'], as_index=False)
      .agg({
          'Max Temp': 'mean',
          'Min Temp': 'mean',
          'Avg Temp': 'mean'
      })
      .rename(columns={
          'Max Temp': 'Mean Max Temp',
          'Min Temp': 'Mean Min Temp',
          'Avg Temp': 'Mean Avg Temp'
      })
)

# Display the resulting dataframe
mean_df.head()

,Station Name,Latitude,Longitude,Elevation,Mean Max Temp,Mean Min Temp,Mean Avg Temp
0,Ada,47.32119,-96.51406,910,54.665667,34.740433,44.703267
1,Adams,48.49988,-98.07588,1580,51.315233,31.839833,41.577767
2,Alamo,48.54652,-103.47186,2157,55.155400,28.167200,41.661500
3,Alexander,47.75056,-103.73358,2202,57.534133,29.601633,43.568067
4,Alvarado,48.24594,-97.02153,809,54.150900,32.517067,43.334233


In [37]:
# Convert DataFrame to SeDF
sedf_ndawn = pd.DataFrame.spatial.from_xy(mean_df, "Longitude", "Latitude", crs=4326)
sedf_ndawn.head()

,Station Name,Latitude,Longitude,Elevation,Mean Max Temp,Mean Min Temp,Mean Avg Temp,SHAPE
0,Ada,47.32119,-96.51406,910,54.665667,34.740433,44.703267,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Adams,48.49988,-98.07588,1580,51.315233,31.839833,41.577767,"{""spatialReference"": {""wkid"": 4326}, ""x"": -98...."
2,Alamo,48.54652,-103.47186,2157,55.155400,28.167200,41.661500,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
3,Alexander,47.75056,-103.73358,2202,57.534133,29.601633,43.568067,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
4,Alvarado,48.24594,-97.02153,809,54.150900,32.517067,43.334233,"{""spatialReference"": {""wkid"": 4326}, ""x"": -97...."


In [39]:
# Define file paths
output_file_path = r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\sedf_ndawn.shp"
output_feature_class = r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\sedf_ndawn_fc"

# Save the SpatialDataFrame to a shapefile
sedf_ndawn.spatial.to_featureclass(location=output_file_path)

# Save the SpatialDataFrame as a feature class in a geodatabase (better for interpolation in ArcGIS)
sedf_ndawn.spatial.to_featureclass(location=output_feature_class)

# Project the shapefile to the desired CRS for interpolation (e.g., EPSG 4326 for geographic coordinates)
output_projected_shapefile = r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\sedf_ndawn_proj.shp"
spatial_reference = arcpy.SpatialReference(4326)

# Project the shapefile for interpolation
arcpy.Project_management(output_file_path, output_projected_shapefile, spatial_reference)

# Reload the projected shapefile as a SpatialDataFrame (optional, for verification or further analysis)
sedf_ndawn_proj = pd.DataFrame.spatial.from_featureclass(output_projected_shapefile)

# Save a projected feature class for easier ArcGIS interpolation
output_projected_feature_class = r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\sedf_ndawn_proj_fc"
arcpy.Project_management(output_feature_class, output_projected_feature_class, spatial_reference)

print(f"Files successfully saved:\n - Original: {output_file_path}\n - Projected: {output_projected_feature_class}")

Files successfully saved:
 - Original: C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\sedf_ndawn.shp
 - Projected: C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\sedf_ndawn_proj_fc


### 2. Interpolate the Weather Data

#### 2a. Interpolation Method 1 - IDW

In [40]:
# Use IDW (Power=2, Variable, 12 pts)
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="sedf_ndawn_proj_fc",
        z_field="mean_avg_temp",
        cell_size=0.021010908,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\IDW_p2V12")

print("IDW was saved to the project.")

IDW was saved to the project.


#### 2b. Interpolation Method 2 - Kriging

In [42]:
# Use Kriging (Ordinary, Gaussian, Lag:0.021011, Variable, 12 pts)
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="sedf_ndawn_proj_fc",
        z_field="mean_avg_temp",
        kriging_model="Gaussian 0.021011 # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\Kriging_sedf1")

print("Kriging was saved to the project")

Kriging was saved to the project


#### 2c. Interpolation Method 3 - Other

In [43]:
# Use Spline (Regularized, weight=0.1, 12 pts)

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb"):
    out_raster = arcpy.sa.Spline(
        in_point_features="sedf_ndawn_proj_fc",
        z_field="mean_avg_temp",
        cell_size=0.021010908,
        spline_type="REGULARIZED",
        weight=0.1,
        number_points=12
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\Lab3_Part2\Lab3_Part2.gdb\Spline_sedf_1")

print("Spline was saved to the project")

Spline was saved to the project


### 3. Comparison